In [8]:
import jsonlines
import glob

from py2neo import Graph, Node, Relationship
from py2neo import NodeMatcher, RelationshipMatcher
import spacy
from tqdm import tqdm
import re

from sentence_transformers import SentenceTransformer
model_path="/home/yuxin.yang/model/moka-ai/m3e-base"
model = SentenceTransformer(model_path)


graph=Graph('http://localhost:7474',user='neo4j',password='cecaceca512',name='neo4j')
graph.delete_all()
node_matcher = NodeMatcher(graph)

data_dir = '/home/yuxin.yang/kg/raw_data/科研数据/**/*.jsonl'
for file_path in glob.glob(data_dir, recursive=True):
    with jsonlines.open(file_path, 'r') as reader:
        #print(file_path)
        for json_obj in reader:
            # 处理或打印每个JSON对象
            #print(json_obj['metadata'])
            existing_node = node_matcher.match("CONTENT",text=json_obj['content']).first()
            if not existing_node:
                vec = model.encode(json_obj['content'], normalize_embeddings=False).tolist()
                content_node=Node("CONTENT", text=json_obj['content'], embedding = vec)
                graph.create(content_node)
                for key, value in json_obj['metadata'].items():
                    #print(f"Key: {key}, Value: {value}")
                    existing_node = node_matcher.match(key,text=value).first()
                    if not existing_node:
                        vec = model.encode(value, normalize_embeddings=False).tolist()
                        ent_node=Node(key,text=value,embedding = vec)


                        graph.create(ent_node)
                        relation=Relationship(content_node,"include",ent_node)
                        graph.create(relation)
                    else:
                        relation=Relationship(content_node,"include",existing_node)
                        graph.create(relation)

